# LlamaIndex Bottoms-Up Development - LLMs and Prompts
This notebook walks through testing an LLM using the primary prompt templates used in llama-index.

## Setup
In this section, we load a test document, create an LLM, and copy prompts from llama-index to test with.

First, let's load a quick document to test with. Right now, we will just load it as plain text, but we can do other operations later!

In [32]:
from pathlib import Path
with open(Path() / ".." / ".." /"data" / "starter_example.md", "r") as f:
    text = f.read()

Next, we create our LLM!

In [33]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

LlamaIndex uses some simple templates under the hood for answering queries -- mainly a `text_qa_template` for obtaining initial answers, and a `refine_template` for refining an existing answer when all the text does not fit into one LLM call.

Let's copy the default templates, and test out our LLM with a few questions.

In [34]:
from llama_index.core import Prompt

text_qa_template = Prompt(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_template = Prompt(
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

Now, lets test a few questions!

## Text QA Template Testing

In [35]:
question = "How can I install llama-index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To install LlamaIndex, you can follow the steps outlined in the [installation](installation.md) guide provided in the context information. The guide suggests cloning the LlamaIndex repository using the command `git clone https://github.com/jerryjliu/llama_index.git` to download the examples folder.


In [36]:
question = "How do I create an index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To create an index, you can follow these steps:

1. Download the LlamaIndex repository and navigate to the `examples/paul_graham_essay` folder.
2. Create a new `.py` file with the provided Python code snippet that builds an index over the documents in the `data` folder.
3. Run the code to build the index.
4. Use the query engine to query the index with a specific question.
5. Optionally, you can view queries and events using logging.
6. To persist the index to disk, use the provided code snippet.
7. To reload the index from disk, use the provided code snippet.


In [37]:
question = "How do I create an index? Write your answer using only code."
prompt = text_qa_template.format(context_str=text, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")

```python
from llama_index

 import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
```

## Refine Template Testing

In [38]:
question = "How do I create an index? Write your answer using only code."
existing_answer = """To create an index using LlamaIndex, you need to follow these steps:

1. Download the LlamaIndex repository by cloning it from GitHub.
2. Navigate to the `examples/paul_graham_essay` folder in the cloned repository.
3. Create a new Python file and import the necessary modules: `VectorStoreIndex` and `SimpleDirectoryReader`.
4. Load the documents from the `data` folder using `SimpleDirectoryReader('data').load_data()`.
5. Build the index using `VectorStoreIndex.from_documents(documents)`.
6. To persist the index to disk, use `index.storage_context.persist()`.
7. To reload the index from disk, use the `StorageContext` and `load_index_from_storage` functions.

Note: This answer assumes that you have already installed LlamaIndex and have the necessary dependencies."""
prompt = refine_template.format(context_msg=text, query_str=question, existing_answer=existing_answer)
response = llm.complete(prompt)
print(response.text)

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist()
```


### Chat Example
The LLM also has a `chat` method that takes in a list of messages, to simulate a chat session. 

In [39]:
from llama_index.core.llms import ChatMessage

chat_history = [
    ChatMessage(role="system", content="You are a helpful QA chatbot that can answer questions about llama-index."),
    ChatMessage(role="user", content="How do I create an index?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant: To create an index, you can follow these general steps:

1. Define the purpose of the index: Determine what specific data or information you want to organize and make easily accessible through the index.

2. Identify the items to be indexed: Decide what items, topics, or categories will be included in the index.

3. Create an index structure: Determine how you want to organize the index, such as alphabetically, chronologically, or by topic.

4. Assign index entries: For each item or topic in your content, create index entries that accurately describe and categorize the information.

5. Compile the index: Gather all the index entries and organize them according to your chosen structure.

6. Format the index: Format the index entries in a consistent and clear manner, making it easy for users to navigate and find information.

7. Review and revise: Double-check the index for accuracy, completeness, and usability. Make any necessary revisions to improve the index's effectiveness

## Conclusion

In this notebook, we covered the low-level LLM API, and tested out some basic prompts with out documentation data.